# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298534828841                   -0.85    5.2    195ms
  2   -8.300216305592       -2.77       -1.26    1.4    111ms
  3   -8.300437175112       -3.66       -1.89    2.6    104ms
  4   -8.300459768950       -4.65       -2.73    2.0    104ms
  5   -8.300464020184       -5.37       -3.16    2.2    113ms
  6   -8.300464368988       -6.46       -3.30    7.8    171ms
  7   -8.300464501974       -6.88       -3.44    1.1   98.5ms
  8   -8.300464577195       -7.12       -3.58    2.0    108ms
  9   -8.300464628461       -7.29       -3.80    1.0   97.4ms
 10   -8.300464639334       -7.96       -3.97    1.0   96.2ms
 11   -8.300464642825       -8.46       -4.25    1.0   96.6ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.66654983748                   -0.71    6.6    510ms
  2   -16.67881380420       -1.91       -1.13    1.4    224ms
  3   -16.67922608450       -3.38       -1.86    2.4    249ms
  4   -16.67927376483       -4.32       -2.70    1.2    221ms
  5   -16.67928566169       -4.92       -3.08    5.1    350ms
  6   -16.67928614527       -6.32       -3.46    1.6    217ms
  7   -16.67928620880       -7.20       -3.97    3.9    269ms
  8   -16.67928622049       -7.93       -4.42    2.9    288ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32174207492                   -0.56    7.4    1.51s
  2   -33.33285742809       -1.95       -1.00    1.0    676ms
  3   -33.33412359468       -2.90       -1.73    4.8    973ms
  4   -33.33426555406       -3.85       -2.64    1.0    662ms
  5   -33.33581145123       -2.81       -2.48    6.4    1.21s
  6   -33.33694118461       -2.95       -2.51    5.2    872ms
  7   -33.33693726389   +   -5.41       -2.52    3.8    849ms
  8   -33.33681601715   +   -3.92       -2.38    1.6    713ms
  9   -33.33684024265       -4.62       -2.43    1.0    661ms
 10   -33.33685786185       -4.75       -2.47    1.0    654ms
 11   -33.33687792626       -4.70       -2.52    1.0    647ms
 12   -33.33688837396       -4.98 

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298674329251                   -0.85    5.5    210ms
  2   -8.300286069929       -2.79       -1.59    1.4   92.1ms
  3   -8.300388891699       -3.99       -2.33    2.5    110ms
  4   -8.300323431766   +   -4.18       -2.17    3.1    131ms
  5   -8.300464092435       -3.85       -3.54    1.0   87.9ms
  6   -8.300464568365       -6.32       -3.79    5.0    197ms
  7   -8.300464636091       -7.17       -4.17    1.0    130ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/yMJC2/src/ase_conversions.jl:100
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32604202278                   -0.56    6.6    1.51s
  2   -33.30082738370   +   -1.60       -1.26    1.0    532ms
  3   -1.635425897586   +    1.50       -0.31    7.1    1.32s
  4   -33.31361322957        1.50       -1.61    6.5    1.18s
  5   -32.97566166171   +   -0.47       -1.20    4.0    909ms
  6   -30.06202977127   +    0.46       -0.81    5.0    968ms
  7   -33.32015991469        0.51       -1.91    4.6    890ms
  8   -33.33371795814       -1.87       -2.15    1.9    670ms
  9   -33.33449142470       -3.11       -2.13    2.5    1.07s
 10   -33.33512439859       -3.20       -2.42    1.8    678ms
 11   -33.33661324435       -2.83       -2.74    1.8    661ms
 12   -33.33664364607       -4.52 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.